In [117]:
# Implement Adaboost using Stumps (1 level Decision Trees)
import math
import random

In [422]:
# Use classes / methods / functions from Decision Tree Implementation

training_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Green', 3, 'Apple'],
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Yellow', 3, 'Lemon'],
    ['Yellow', 4, 'Lemon'],
    ['Yellow', 4, 'Lemon'],
    ['Yellow', 4, 'Lemon'],
]

header = ["color", "diameter", "label"]

In [210]:
# Get unique values of a column in a dataset
def unique_vals(rows, col):
    return set([row[col] for row in rows])

def class_counts(rows):
    counts = {}
    for row in rows:
        label = row[-1]
        if label not in counts.keys():
            counts[label] = 1
        else:
            counts[label] += 1
    return counts

def is_numeric(value):
    """Test if a value is numeric."""
    return isinstance(value, int) or isinstance(value, float)



In [323]:
class Question:
    """A Question is used to partition a dataset.

    This class just records a 'column number' (e.g., 0 for Color) and a
    'column value' (e.g., Green). The 'match' method is used to compare
    the feature value in an example to the feature value stored in the
    question. See the demo below.
    """

    def __init__(self, column, value):
        self.column = column
        self.value = value
        
    # Takes an example row and sees if it matches the question at hand
    def match(self, example):
        # Compare the feature value in an example to the
        # feature value in this question.
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        # This is just a helper method to print
        # the question in a readable format.
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))

In [310]:
def partition(rows, question):
    """Partitions a dataset.

    For each row in the dataset, check if it matches the question. If
    so, add it to 'true rows', otherwise, add it to 'false rows'.
    """
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

def gini(rows):
    """"Calculate the Gini Impurity for a list of rows.

    There are a few different ways to do this, I thought this one was
    the most concise. See:
    https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity
    """
    counts = class_counts(rows) # dictionary
    impurity = 1
    for lbl, count in counts.items():
        prob_of_lbl = count / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

def info_gain(left, right, current_uncertainty):
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - (p * gini(left) + (1-p) * gini(right))

def find_best_split(rows):
    """Find the best question to ask by iterating over every feature / value
    and calculating the information gain."""
    
    best_question = None
    best_gain = 0
    
    current_uncertainty = gini(rows)
    col_num = 0
    last_feature_col_num = len(rows[0]) - 2
    
    while col_num <= last_feature_col_num:
        unique = unique_vals(rows, col_num)
        for val in unique:
            # set the question to ask
            q = Question(col_num, val)
            # partition based on the question
            true_rows, false_rows = partition(rows, q)
            
            # if it doesn't divide data at all, move on to next value
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue
                
            # find information gain
            ig = info_gain(true_rows, false_rows, current_uncertainty)
            if ig >= best_gain:
                best_gain = ig
                best_question = q     
                
        col_num += 1
        
    return best_gain, best_question

In [414]:
class Stump:
    
    def __init__(self, data):
        self.data = data
        self.best_question = ""
        self.true_rows = ""
        self.false_rows = ""
        self.true_label = ""
        self.false_label = ""
        
    def predictions(self):
        best_gain, best_question = find_best_split(self.data)
        self.best_question = best_question
        true_rows, false_rows = partition(self.data, self.best_question)
        self.true_rows, self.false_rows = true_rows, false_rows

        # predict majority class for true branch and false branch

        # sort true branch
        sorted_true = sorted(class_counts(true_rows).items(), key=lambda x: x[1], reverse=True)
        true_label = sorted_true[0][0]

        # sort false branch
        sorted_false = sorted(class_counts(false_rows).items(), key=lambda x: x[1], reverse=True)
        false_label = sorted_false[0][0]

        print("Prediction based on: ", best_question)

        print("True prediction: ", true_label)
        print("False prediction: ", false_label)
        
        self.true_label = true_label
        self.false_label = false_label
        
        return true_label, false_label

In [214]:
# Example
stump = Stump(training_data)

In [215]:
stump.predictions()

Prediction based on:  Is diameter >= 3?
True prediction:  Apple
False prediction:  Grape


('Apple', 'Grape')

In [216]:
stump.true_rows

[['Green', 3, 'Apple'],
 ['Yellow', 3, 'Apple'],
 ['Yellow', 3, 'Lemon'],
 ['Green', 3, 'Apple'],
 ['Yellow', 3, 'Apple'],
 ['Green', 3, 'Apple'],
 ['Yellow', 3, 'Apple'],
 ['Green', 3, 'Apple'],
 ['Yellow', 3, 'Apple'],
 ['Green', 3, 'Apple'],
 ['Yellow', 3, 'Apple'],
 ['Green', 3, 'Apple'],
 ['Yellow', 3, 'Apple'],
 ['Green', 3, 'Apple'],
 ['Green', 3, 'Apple'],
 ['Yellow', 3, 'Apple'],
 ['Yellow', 3, 'Apple'],
 ['Yellow', 3, 'Lemon'],
 ['Yellow', 4, 'Lemon'],
 ['Yellow', 4, 'Lemon'],
 ['Yellow', 4, 'Lemon']]

In [217]:
stump.false_rows

[['Red', 1, 'Grape'],
 ['Red', 1, 'Grape'],
 ['Red', 1, 'Grape'],
 ['Red', 1, 'Grape'],
 ['Red', 1, 'Grape'],
 ['Red', 1, 'Grape'],
 ['Red', 1, 'Grape'],
 ['Red', 1, 'Grape']]

In [415]:
# Helper Functions
def insert_weights(data):
    for row in data:
        row.insert(0, 1 / len(data))

    return data

def calculate_error(true_rows, false_rows, true_label, false_label):
    
    num_wrong = 0

    for row in true_rows:
        if row[-1] != true_label:
            num_wrong += 1
            row.insert(0, "wrong")
        else:
            row.insert(0, "right")

    for row in false_rows:
        if row[-1] != false_label:
            num_wrong += 1
            row.insert(0, "wrong")
        else:
            row.insert(0, "right")

    total_error = num_wrong / len(training_data)

    return total_error

def calculate_performance(error, alpha=0.5):
    # find performance of stump using total error
    try:
        performance = alpha * math.log((1 - error) / error)
    except:
        performance = 1
    return performance

def update_and_normalize(data, performance):
    # updating weights (based on 'right' or 'wrong')
    for row in data:
        if row[0] == "right":
            row[1] = row[1] * math.exp(-performance)
        else:
            row[1] = row[1] * math.exp(performance)
            
    # normalize weights
    total_weight = 0
    for row in data:
        total_weight += row[1]

    for row in data:
        row[1] = row[1] / total_weight

    return data


def create_buckets(data):
    # create buckets for new dataset (need to use indices)
    for i in range(len(data)):
        if i == 0:
            data[i].insert(0, data[i][1])
        else:
            data[i].insert(0, data[i-1][0] + data[i][1]) # previous bucket + current weight

    return data

def populate_new_data(data):
    # populate new dataset based on buckets in previous dataset
    new_data = []
    for i in range(len(data)):
        rand_num = random.uniform(0, 1)
        # find row to append to new_data
        for row in data:
            if rand_num < row[0]: # if less than bucket number
                new_data.append(row)
                break

    return new_data # should have more rows of "wrong" instances

def remove_first_3(data):
    # remove the first 3 cols for new stump creation
    for i in range(len(data)):
        data[i] = data[i][3:]

    return data


In [423]:
class AdaBoost:
    
    def __init__(self, n_estimators=11):
        self.n_estimators = n_estimators
        
    def fit(self, header, data):
        """
        Keeps passing in new data to the Stump classifier for 'n_estimators' number of times
        Returns a list of stumps
        """
        stumps = []
        
        for i in range(self.n_estimators):
            print("Step:", i+1)
            s = Stump(data)
            true_label, false_label = s.predictions()
            
            # append stump here
            stumps.append(s)
            
            # prepare data for next stump
            data = insert_weights(data)
            error = calculate_error(s.true_rows, s.false_rows, true_label, false_label)
            performance = calculate_performance(error)
            data = s.true_rows + s.false_rows
            data = update_and_normalize(data, performance)
            data = create_buckets(data)
            data = populate_new_data(data)
            data = remove_first_3(data)
            
        return stumps
            
    
    def predict_instance(self, stumps, instance):
        stump_predictions = []
        for stump in stumps:
            instance_match = stump.best_question.match(instance)
            if instance_match: # split criterion is True
                stump_predictions.append(stump.true_label)
            else:
                stump_predictions.append(stump.false_label)
        
        # dictionary to store predictions + counts
        pred_dict = {}
        for pred in stump_predictions:
            if pred not in pred_dict.keys():
                pred_dict[pred] = 1
            else:
                pred_dict[pred] += 1
        
        max_key = max(pred_dict, key=pred_dict.get)
        return max_key
    
    def predict(self, stumps, test_rows):
        predictions = []
        for row in test_rows:
            prediction = self.predict_instance(stumps, row)
            predictions.append(prediction)
            
        return predictions
    
    def accuracy(self, test_rows, predictions):
        # assuming the label is in the last column of test rows
        labels = [row[-1] for row in test_rows]
        correct = 0
        for i in range(len(labels)):
            if labels[i] == predictions[i]:
                correct += 1
        
        accuracy_score = correct / len(labels)
        return accuracy_score        
        
        

In [424]:
ada = AdaBoost()

In [425]:
stumps = ada.fit(header, training_data)

Step: 1
Prediction based on:  Is diameter >= 3?
True prediction:  Apple
False prediction:  Grape
Step: 2
Prediction based on:  Is diameter >= 4?
True prediction:  Lemon
False prediction:  Apple
Step: 3
Prediction based on:  Is diameter >= 3?
True prediction:  Lemon
False prediction:  Grape
Step: 4
Prediction based on:  Is diameter >= 3?
True prediction:  Apple
False prediction:  Grape
Step: 5
Prediction based on:  Is color == Yellow?
True prediction:  Lemon
False prediction:  Apple
Step: 6
Prediction based on:  Is diameter >= 3?
True prediction:  Lemon
False prediction:  Grape
Step: 7
Prediction based on:  Is diameter >= 4?
True prediction:  Lemon
False prediction:  Apple
Step: 8
Prediction based on:  Is color == Red?
True prediction:  Grape
False prediction:  Apple
Step: 9
Prediction based on:  Is diameter >= 3?
True prediction:  Lemon
False prediction:  Grape
Step: 10
Prediction based on:  Is diameter >= 3?
True prediction:  Apple
False prediction:  Grape
Step: 11
Prediction based on

In [426]:
# Example
instances = [['Yellow', 3, 'Apple'],
             ['Red', 1, 'Grape']
            ]
pred = ada.predict(stumps, instances)
pred

['Apple', 'Grape']

In [427]:
accuracy = ada.accuracy(instances, pred)
accuracy

1.0

In [428]:
# Try Titanic data
import pandas as pd

In [676]:
df = pd.read_csv('titanic_train.csv')
to_drop = ['PassengerId', 'Cabin', 'Ticket', 'Name']
df = df.drop(to_drop, axis=1)
df.head(20)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
5,0,3,male,NaN,0,0,8.4583,Q
6,0,1,male,54.0,0,0,51.8625,S
7,0,3,male,2.0,3,1,21.0750,S
8,1,3,female,27.0,0,2,11.1333,S
9,1,2,female,14.0,1,0,30.0708,C


In [677]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


In [678]:
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  712 non-null    int64  
 1   Pclass    712 non-null    int64  
 2   Sex       712 non-null    object 
 3   Age       712 non-null    float64
 4   SibSp     712 non-null    int64  
 5   Parch     712 non-null    int64  
 6   Fare      712 non-null    float64
 7   Embarked  712 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 50.1+ KB


In [679]:
numeric_cols = ['Pclass', 'SibSp', 'Parch']
for col in numeric_cols:
    df[col] = df[col].astype(float)

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  712 non-null    int64  
 1   Pclass    712 non-null    float64
 2   Sex       712 non-null    object 
 3   Age       712 non-null    float64
 4   SibSp     712 non-null    float64
 5   Parch     712 non-null    float64
 6   Fare      712 non-null    float64
 7   Embarked  712 non-null    object 
dtypes: float64(5), int64(1), object(2)
memory usage: 50.1+ KB


In [680]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3.0,male,22.0,1.0,0.0,7.2500,S
1,1,1.0,female,38.0,1.0,0.0,71.2833,C
2,1,3.0,female,26.0,0.0,0.0,7.9250,S
3,1,1.0,female,35.0,1.0,0.0,53.1000,S
4,0,3.0,male,35.0,0.0,0.0,8.0500,S


In [681]:
# move 'Survived' col to end
survived = df.iloc[:,0]
df = df.iloc[:,1:]
df = pd.concat([df, survived], axis=1)
df.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Survived
0,3.0,male,22.0,1.0,0.0,7.2500,S,0
1,1.0,female,38.0,1.0,0.0,71.2833,C,1
2,3.0,female,26.0,0.0,0.0,7.9250,S,1
3,1.0,female,35.0,1.0,0.0,53.1000,S,1
4,3.0,male,35.0,0.0,0.0,8.0500,S,0


In [682]:
train_set = df.iloc[:500,:]
test_set = df.iloc[500:,:]
train_set.shape

(500, 8)

In [683]:
# convert train_set to list of lists
train_rows = []
for i in range(train_set.shape[0]):
    row = []
    for element in train_set.iloc[i]:
        row.append(element)
    train_rows.append(row)

train_rows[:5]

[[3.0, 'male', 22.0, 1.0, 0.0, 7.25, 'S', 0],
 [1.0, 'female', 38.0, 1.0, 0.0, 71.2833, 'C', 1],
 [3.0, 'female', 26.0, 0.0, 0.0, 7.925, 'S', 1],
 [1.0, 'female', 35.0, 1.0, 0.0, 53.1, 'S', 1],
 [3.0, 'male', 35.0, 0.0, 0.0, 8.05, 'S', 0]]

In [684]:
# convert test_set to list of lists
test_rows = []
for i in range(test_set.shape[0]):
    row = []
    for element in test_set.iloc[i]:
        row.append(element)
    test_rows.append(row)

test_rows[:5]

[[3.0, 'female', 9.0, 3.0, 2.0, 27.9, 'S', 0],
 [2.0, 'female', 28.0, 0.0, 0.0, 13.0, 'S', 1],
 [3.0, 'male', 32.0, 0.0, 0.0, 7.925, 'S', 0],
 [2.0, 'male', 31.0, 1.0, 1.0, 26.25, 'S', 0],
 [3.0, 'female', 41.0, 0.0, 5.0, 39.6875, 'S', 0]]

In [685]:
header = list(train_set.columns)
header

['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Survived']

In [686]:
# Train with AdaBoost classifier
ada = AdaBoost(n_estimators=21)
stumps = ada.fit(header, train_rows)

Step: 1
Prediction based on:  Is Sex == female?
True prediction:  1
False prediction:  0
Step: 2
Prediction based on:  Is Pclass >= 3.0?
True prediction:  0
False prediction:  1
Step: 3
Prediction based on:  Is Pclass >= 3.0?
True prediction:  1
False prediction:  0
Step: 4
Prediction based on:  Is Fare >= 52.5542?
True prediction:  1
False prediction:  1
Step: 5
Prediction based on:  Is Fare >= 52.5542?
True prediction:  0
False prediction:  0
Step: 6
Prediction based on:  Is Fare >= 52.5542?
True prediction:  1
False prediction:  1
Step: 7
Prediction based on:  Is Age >= 9.0?
True prediction:  0
False prediction:  1
Step: 8
Prediction based on:  Is Parch >= 2.0?
True prediction:  0
False prediction:  1
Step: 9
Prediction based on:  Is Fare >= 89.1042?
True prediction:  1
False prediction:  0
Step: 10
Prediction based on:  Is Age >= 26.0?
True prediction:  1
False prediction:  0
Step: 11
Prediction based on:  Is Age >= 24.0?
True prediction:  0
False prediction:  1
Step: 12
Prediction

In [687]:
# Predict
predictions = ada.predict(stumps, test_rows)

In [688]:
accuracy = ada.accuracy(test_rows, predictions)
accuracy

0.7594339622641509